# Tugas Besar 1 CNN

## Eksperimen dengan split train dan test

In [1]:
# Import dependencies
from cnn.cnn import MyCnn
from cnn.layers import Conv2D, Pooling, Dense, Flatten
from typing import List
import numpy as np
import tensorflow as tf
import pickle
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

In [2]:
# Constant
IMG_HEIGHT, IMG_WIDTH = 64, 64
TRAIN_IMG_DIR = './dataset/train'
TEST_IMG_DIR = './dataset/test'
SEED = 6459164
SIGMOID_THRESHOLD = 0.5

In [3]:
# Helper Functions
def save_model(model, namefile: str):
    model_file = open('cnn_model', 'ab')
    pickle.dump(model, model_file)
    model_file.close()


def load_model(namefile: str):
    model_file = open('cnn_model', 'rb')
    model = pickle.load(model_file)
    model_file.close()

    return model


def load_img_as_train_dataset() -> np.array:
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        TRAIN_IMG_DIR,
        seed=SEED,
        image_size=(IMG_HEIGHT, IMG_WIDTH)
    )

    imgs = []
    labels = []
    for img_batch, label_batch in train_ds:
        imgs.append(img_batch.numpy())
        labels.append(label_batch.numpy())

    imgs = np.concatenate(imgs)
    labels = np.concatenate(labels)

    return imgs, labels


def load_img_as_test_dataset() -> np.array:
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
        TEST_IMG_DIR,
        seed=SEED,
        image_size=(IMG_HEIGHT, IMG_WIDTH)
    )

    imgs = []
    labels = []
    for img_batch, label_batch in test_ds:
        imgs.append(img_batch.numpy())
        labels.append(label_batch.numpy())

    imgs = np.concatenate(imgs)
    labels = np.concatenate(labels)

    return imgs, labels


def normalize_img(img: np.array) -> np.array:
    scalev = np.vectorize(lambda x: x / 255.0)
    return scalev(img)


def reorganize_layer(img: np.array) -> List[np.array]:
    input_shape = img.shape
    rows = input_shape[0]
    cols = input_shape[1]
    depth = input_shape[2]

    layers = [np.array([[0] * cols] * rows) for _ in range(depth)]

    for i, row in enumerate(img):
        for j, col in enumerate(row):
            for k, val in enumerate(col):
                layers[k][i][j] = val

    return layers


def interpret_class(res: List[np.array]) -> int:
    return 0 if res[0][0] < SIGMOID_THRESHOLD else 1

In [4]:
# Load Images
imgs, labels = load_img_as_train_dataset()
labels = list(labels)

img_list = []
for img in imgs:
    img_norm = normalize_img(img)
    img_reorganized = reorganize_layer(img)
    img_list.append(img_reorganized)

Found 181 files belonging to 2 classes.


In [5]:
# Training
kf = KFold(n_splits=10, random_state=SEED, shuffle=False)
scores = []

img_list = np.array(img_list)
labels = np.array(labels)

for train_index, test_index in kf.split(img_list):
    # Model
    cnn = MyCnn()
    cnn.add(Conv2D(0, 4, np.array([3, 3]), 1, np.array(
        [[IMG_HEIGHT, IMG_WIDTH] for _ in range(3)])))
    cnn.add(Pooling(np.array([2, 2]), 2, 'max'))
    cnn.add(Flatten())
    cnn.add(Dense(1, 'sigmoid'))

    # Data
    x_train = img_list[train_index]
    y_train = labels[train_index]
    
    x_test = img_list[test_index]
    y_test = labels[test_index]

    cnn.fit(x_train, y_train, 1, 100, 0.1, 0.01)

    # Predict
    res_classes = []
    for img, label in zip(x_test, y_test):
        res, layers_input = cnn.feed_forward(img)
        res_class = interpret_class(res)
        res_classes.append(res_class)

    print(res_classes, flush=True)
    print(accuracy_score(y_test, res_classes), flush=True)
    scores.append(accuracy_score(y_test, res_classes))

Epoch: 0..
[1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]
0.6842105263157895
Epoch: 0..
[0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0]
0.3888888888888889
Epoch: 0..
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
0.5555555555555556
Epoch: 0..
[0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
0.7222222222222222
Epoch: 0..
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
0.6111111111111112
Epoch: 0..
[1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1]
0.4444444444444444
Epoch: 0..
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.5
Epoch: 0..
[0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]
0.5
Epoch: 0..
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.5
Epoch: 0..
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
0.5


In [7]:
# Print scores
average_score = np.mean(scores)
print(average_score)

0.5406432748538011
